# Le problème à résoudre

Vous avez été embauché comme consultant chez *Schmurfvengo* qui vient d'obtenir de façon inespérée la gestion des vélibs à Paris. Le problème est le suivant : pour cause de soucis techniques, il faut régulièrement remplacer l'ensemble de la flotte (1300 vélos) en **une nuit**. 

Pour cela, vous disposez de 4 gros camions qui ont chacun la possibilité de livrer 500 vélos au cours de la nuit. Cependant, pour éviter de polluer la capitale, ces camions se contentent de livrer chacun dans une station (la plus proche de l'entrepot correspondant). Les sations correspondant aux 4 camions sont :

 * Pierre Ginier Clichy
 * Le Vau - Maurice Bertaux
 * Le Brix et Mesmin-Jourdan
 * Siam - La Pompe
 
Il y a 68 stations en fonctionnement dans la capitale. Il faut donc acheminer les vélos aux autres stations. Pour cela on dispose de navettes électriques pouvant se déplacer entre les stations de vélos, sur de courtes distances. Une navette peut livrer 50 vélos au cours de la nuit. On veut livrer des vélos dans toutes les stations sans dépasser 80% de leur capacité (pour laisser quelques places vides).  Votre travail est d'effectuer une étude rapide préalable pour estimer une solution possible d'acheminement de l'ensemble des vélos (combien de navettes placer, où, etc). On considèrera que deux stations peuvent être réliées par une navette si elles sont distantes au maximum de 2500 mètres à vol d'oiseau.

Exemple : la station "Institut de France" a une capacité de 17 vélos, on veut lui donc lui en livrer 13 : on pourra emprunter la navette qui va de "Pierre Ginier Clichy" à "Hauteville - Bonne Nouvelle" puis une seconde navette de "Hauteville - Bonne Nouvelle" à "Institut de France". (Remarque, les 13 vélos peuvent aussi venir de plusieurs navettes différentes arrivant à "Institut de France")

## Les données

Pour résoudre ce problème, vous pouvez utiliser les données de [Open Data Paris](https://opendata.paris.fr). La table dont vous avez besoin est [Stations Velibs : emplacement des stations](https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/). Le fichier *json* est disponible dans ce dossier, les données sont directement interprétables en python de cette façon.

In [1]:
import json
data = json.load(open("media/velib-emplacement-des-stations.json"))

La variable `data` est une liste dont chaque élément est un dictionnaire python contenant les informations d'une station vélib.

In [2]:
len(data)

Voilà par exemple, le premier élément de la liste, la station *Chernoviz - Raynouard*

In [3]:
data[0]

In [4]:
data[0]["fields"]["capacity"]

On vous fourni la fonction suivante, qui permet de calculer la distance en mètres entre deux points données sous forme `(latitude, longitude)`.

In [5]:
import math
def toRad(angle):
    """
    Converti un angle donné en degré en un angle donné en radian.
    
    INPUT:
        
        - angle, un angle en degré
        
    OUTPUT : la valeur en radian
    """
    return angle*2*math.pi/360

def distance(p1, p2):
    """
    Renvoie la distance en mètres entre les points `p1` et `p2` données en coordonnées géographique
    
    INPUT:
    
        - p1, un tuple (latitude, longitude)
        - p2, un tuple (latitude, longitude)
        
    OUPUT : la distance à vol d'oiseau en mètres entre p1 et p2
    """
    lat1, lon1 = p1
    lat2, lon2 = p2
    R = 6371e3
    phi1 = toRad(lat1)
    phi2 = toRad(lat2)
    dphi = toRad(lat2 - lat1)
    dlam = toRad(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlam/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R*c

Par exemple, la distance entre la première station de la liste *Chernoviz - Raynouard* et la seconde, *Choron - Martyrs* est légèrement inférieure à 5km.

In [6]:
distance((data[0]["fields"]["lat"], data[0]["fields"]["lon"]), (data[1]["fields"]["lat"], data[1]["fields"]["lon"])) 

On vous mondre aussi les scripts suivants qui permettent de faire un affichage graphique avec *mathplotlib*. (Le premier affiche des points et le second des lignes)

In [7]:
import matplotlib.pyplot as plt

x = [d["fields"]["lon"] for d in data]
y = [d["fields"]["lat"] for d in data]


plt.scatter(x,y,s=5)

plt.title('Les stations velibs dans Paris')
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.savefig('StationsVelib.png')
plt.show()

In [8]:
import matplotlib.pyplot as plt


station_names = set(d['fields']['name'] for d in data)
station_dict = {d['fields']['name']:d for d in data}
station = station_names.pop()

while len(station_names) > 0:
    next_station = min(station_names, key = lambda n: distance((station_dict[station]["fields"]["lat"], station_dict[station]["fields"]["lon"]), (station_dict[n]["fields"]["lat"], station_dict[n]["fields"]["lon"])))
    station_names.remove(next_station)
    plt.plot([station_dict[station]["fields"]["lon"], station_dict[next_station]["fields"]["lon"]], [station_dict[station]["fields"]["lat"], station_dict[next_station]["fields"]["lat"]], 'k-', lw=1)
    station = next_station

plt.title('Un chemin reliant toutes les stations')
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.savefig('CheminsStations.png')
plt.show()

## A vous de jouer !

Vous avez maintenant tous les éléments pour résoudre le problème. Petit résumé.

**Le but** : acheminer le maximum de vélos sur les 1300.

**Les contraintes** :

 * respecter la capacité de chaque station (pas plus de 80% de remplissage)
 * respecter les contraintes de l'énoncé sur les camions de départ et les navettes.
 
**Le résultat** : votre résultat doit être facilement utilisable. En particulier vous devez implanter les foncion `navette` et `remplissage` définie en bas de la fiche et donner les lignes de codes affichant les réponses aux questions. 

**Un peu de dessin ?** Donnez une illustration de votre résultat en utilisant matplotlib.

**Comment faire ?** Aidez-vous des fiches intermédiaires proposées, discutez avec votre groupe, posez des questions, lisez wikiedia.

In [19]:
# F = ???


In [21]:
def navette(flot, station1, station2):
    """
    S'il existe une navette entre `station1` et `station2`, renvoie le nombre de vélos qui seront transportés par cette navette durant la nuit. Sinon, renvoie 0.
    
    INPUT:
    
        - flot, l'objet contenant le résultat de vos calculs
        - station1, le nom d'une station Velib
        - station2, le nom d'une seconde station Vélib
    """
    # à compléter


In [23]:
# par exemple...
navette(F, "Pierre Ginier Clichy", "Hauteville - Bonne Nouvelle") # pour mes calculs 50 vélos

In [24]:
# vérifons qu'on ne dépasse pas la capacité des navettes
assert all( navette(F, d1["fields"]["name"], d2["fields"]["name"]) <= 50 for d1 in data for d2 in data)

In [26]:
# vérifons qu'on n'a pas de navettes entre une station et elle-même
assert all( navette(F, d["fields"]["name"], d["fields"]["name"]) == 0 for d in data)

In [27]:
def remplissage(flot, station):
    """
    Renvoie le nombre de vélos qu'on aura installé à `station` à la fin de la nuit.
    
    INPUT :
    
        - flot, l'objet contenant le résultat de vos calculs
        - station, le nom d'une station velib
    """
    # à compléter


In [35]:
# par exemple...
remplissage(F, "Hauteville - Bonne Nouvelle") # pour mes calculs 13

In [29]:
# vérifons qu'on ne dépasse pas la capacité des stations
assert all( remplissage(F, d["fields"]["name"]) <= 0.8 * d["fields"]["capacity"] for d in data)

In [30]:
# vérifons qu'on ne distribue pas des "demi-vélos"
assert all( remplissage(F, d["fields"]["name"]) == int(remplissage(F, d["fields"]["name"])) for d in data)

**Combien de vélos avez-vous livré ?**

Répondez par une ligne de code qui affiche la réponse en se basant sur vos calculs.

**Combien de navettes avez-vous utilisé ?**

## Un peu de dessin ?

On voudrait se faire une idée du réseau de navette, pouvez-vous le représenter graphiquement ?